<a href="https://colab.research.google.com/github/champ-rtu/Project2/blob/main/Ironny_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#Import Libraty 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import csv

**There are 6 section in this project**
1. Import data
2. Preprocessing 
3. Numerical representation
4. Oversampling
5. Train the data by machine learning models
6. Evaluation in Test dataset

#**Import text data** 

Train and validataion data are imported in this section
there are two data in this project which is raw text and label data




In [4]:
#Read raw text data and label data
#Open train_text file 
x = []
x_t = []
with open('train_text.txt', 'r', encoding='utf-8') as f:
  data = f.readlines()
  
  for row in data:
    x.append(row)
    x_t.append(row)

#Open train_label file 
y = []
y_t = []
with open('train_labels.txt', 'r', encoding='utf-8') as f:
  data = f.read().split('\n')

  for row in data:
    y.append(row)
    y_t.append(row)



In [ ]:
#Read raw text data and label data
#Open val_text file 
x_val = []
with open('val_text.txt', 'r', encoding='utf-8') as f:
  data = f.readlines()
  
  for row in data:
    x.append(row)
    x_val.append(row)

#Open val_label file 
y_val = []
with open('train_labels.txt', 'r', encoding='utf-8') as f:
  data = f.read().split('\n')

  for row in data:
    y.append(row)
    y_val.append(row)

In [5]:
#combine both data to dataframe
data = pd.DataFrame(zip(x, y),  columns=['Comments', 'Labels'] )
data.head()

,Comments,Labels
0,seeing ppl walking w/ crutches makes me really...,1
1,"look for the girl with the broken smile, ask h...",0
2,Now I remember why I buy books online @user #s...,1
3,@user @user So is he banded from wearing the c...,1
4,Just found out there are Etch A Sketch apps. ...,1


In [6]:
# check the number  of data
data.shape

(2862, 2)

In [7]:
#Identify label data
# It is a slight unbalanced class dataset
data['Labels'].value_counts()

1    1445
0    1417
Name: Labels, dtype: int64

# **Preprocessing**
For preprocessing. the process consists of 
1. tokenize
2. lowercase transformation
3. Remove punctuation and stopword
4. Steming

In [8]:
#import library for preprocessing step
import nltk
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import string
import re
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
#create punctuation, stopword and stem 
punct = string.punctuation #create list of punctuation
stopwords = list(STOP_WORDS) #create list of stopword
nlp = spacy.load('en_core_web_sm')
ps = PorterStemmer() #steming
STOP_WORDS.add('user') #because the text data contain a lot of user. So "user" is added in stopword.
nlp.vocab['user'].is_stop # confirm that user is stopword 

True

In [10]:
#Create fucction for preprocessing step
#the first function is remove punctuation, covert text to lower case
def preprocessing_1(text):
  txt = re.sub(r'\d+','', text)
  txt = txt.translate(str.maketrans("","",punct))
  txt = txt.lower()
  txt = txt.strip()
  return txt

#For the second function; tokenization and remove stopwords
def preprocessing_2(text):
  txt = word_tokenize(text)
  txt = [i for i in txt if not i in stopwords]
  txt = txt = " ".join(txt)
  return txt

#The last function; stemming
def stem(text):
  txt = word_tokenize(text)
  txt = [ps.stem(i) for i in txt ]
  txt = txt = " ".join(txt)
  return txt


In [11]:
#Apply preprocessing function to the dataset
data['Comments'] = data['Comments'].apply(lambda x: preprocessing_1(x))
data['Comments'] = data['Comments'].apply(lambda x: preprocessing_2(x))
data['Comments'] = data['Comments'].apply(lambda x: stem(x))

data.head()

,Comments,Labels
0,see ppl walk w crutch make excit week life,1
1,look girl broken smile ask want stay love 💕🎵,0
2,rememb buy book onlin user servicewithasmil,1
3,user user band wear cloth karma,1
4,found etch sketch app oldschool notoldschool,1


# **Feature exaction section**
In the process, two feature extraction method were selected 
  1. CounterVectorizer
  2. TfidfVectorizer

In [12]:
#import importance library for feature extraction method
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier


from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

In [13]:
#split dataset to x and y variables
y = data['Labels']
x = data['Comments']
#applied TF-IDF approach
tf=TfidfVectorizer() 
x_tf = tf.fit_transform(x)

In [14]:
#applied CountVectorizer
ct = CountVectorizer()
x_ct = ct.fit_transform(x)

In [15]:
#Split train and test dataset tf-idf approach
X_train, X_test, y_train, y_test =  train_test_split(x_tf, y, test_size = 0.3,
                                                     random_state =0, shuffle = True)
#Split train and test dataset couter vectorizer approach
X_train_c, X_test_c, y_train_c, y_test_c =  train_test_split(x_ct, y, test_size = 0.3, 
                                                             random_state =0, shuffle = True)

# **Oversampling approach**

In [16]:
#import library for oversampling
from imblearn.over_sampling import RandomOverSampler

os =  RandomOverSampler( )
X_sm, y_sm = os.fit_sample(X_train, y_train) #apply Oversampling to x train with TFIDF method 
Xc_sm, yc_sm = os.fit_sample(X_train_c, y_train_c) #apply Oversampling to x train with CounterVectorizer method 

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [17]:
#Check dimension after oversampling approach
X_sm.shape

(2058, 6517)

# **Train data**
Three machine learning algorithms are selected for data training 
1. Logistic regression
2. Support vector machine
3. Randomforest classifier

**Logistic regression**

In [19]:
#Import library
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [20]:
#Create logistic regression model
log_class=LogisticRegression()
# created dict which use for Gridsearch
grid={'C':10.0 **np.arange(-3,4),'penalty':['l1','l2']}

In [21]:
#Applied Gridsearch for parameter tuning and train with TF-IDF as input
log=GridSearchCV(estimator=log_class,param_grid=grid,cv=5,n_jobs=-1,scoring='f1_macro')
log.fit(X_sm,y_sm)

GridSearchCV(cv=5, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                         'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1_macro', verbose=0)

In [22]:
#Applied Gridsearch for parameter tuning and train with Counter vector as input
log_c=GridSearchCV(estimator=log_class,param_grid=grid,cv=5,n_jobs=-1,scoring='f1_macro')
log_c.fit(Xc_sm,yc_sm)

GridSearchCV(cv=5, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                         'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1_macro', verbose=0)

Evaluation training set

In [23]:
#import library for evaluation
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix,f1_score

In [24]:
#Prediction and Evaluation the model with TF-IDF as input
y_pred_log=log.predict(X_test)
print(classification_report(y_test,y_pred_log))

              precision    recall  f1-score   support

           0       0.67      0.67      0.67       443
           1       0.64      0.64      0.64       416

    accuracy                           0.66       859
   macro avg       0.66      0.65      0.66       859
weighted avg       0.66      0.66      0.66       859



In [25]:
#Prediction and Evaluation the model with CounterVectorizer as input
y_pred_logc=log_c.predict(X_test_c)
print(classification_report(y_test,y_pred_logc))

              precision    recall  f1-score   support

           0       0.65      0.62      0.64       443
           1       0.62      0.64      0.63       416

    accuracy                           0.63       859
   macro avg       0.63      0.63      0.63       859
weighted avg       0.63      0.63      0.63       859



**Support vector machine**

In [26]:
#Import and create svm models for train different input (TFIDF, count vector)
from sklearn import svm
#Fit with TF-IDF
svc1 = svm.LinearSVC(C=10)
#Fit with Countervectorizer
svc2 = svm.LinearSVC(C=10)

In [27]:
#Training data 
svc1.fit(X_sm,y_sm) #TF-IDF method
svc2.fit(Xc_sm,yc_sm) #CounterVectorizer approach


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=5, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [ ]:
#Evaluation in training set

In [30]:
#Prediction and Evaluation the model with TF-IDF as input for training phase
y_pred=svc1.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.61      0.58      0.60       443
           1       0.58      0.61      0.59       416

    accuracy                           0.60       859
   macro avg       0.60      0.60      0.60       859
weighted avg       0.60      0.60      0.60       859



In [31]:
#Prediction and Evaluation the model with CounterVectorizer as input for training phase
y_pred_c=svc2.predict(X_test_c)
print(classification_report(y_test,y_pred_c))

              precision    recall  f1-score   support

           0       0.63      0.61      0.62       443
           1       0.60      0.61      0.60       416

    accuracy                           0.61       859
   macro avg       0.61      0.61      0.61       859
weighted avg       0.61      0.61      0.61       859



**Randomforest Classifier**

In [32]:
#import library and create Randomforest model
from sklearn.ensemble import RandomForestClassifier
#Create models
clf_forest = RandomForestClassifier()

In [34]:
# Number of trees in random forest
n_estimators = [50,100,200]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [35]:
#import library and create Randomforest model
from sklearn.model_selection import RandomizedSearchCV
rf_random = RandomizedSearchCV(estimator = clf_forest, 
                               param_distributions = random_grid, 
                               n_iter = 100, cv = 3, verbose=2, 
                               random_state=42, n_jobs = -1)
rf_random.fit(X_sm,y_sm) #Train model with TF-IDF input

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  6.5min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [36]:
rf_random_c = RandomizedSearchCV(estimator = clf_forest, 
                               param_distributions = random_grid, 
                               n_iter = 100, cv = 3, verbose=2, 
                               random_state=42, n_jobs = -1)
rf_random_c.fit(Xc_sm,yc_sm) #Train model with Count vector input

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  6.2min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

Evaluation in training set

In [37]:
#Prediction and Evaluation the model with TF-IDF as input for training phase
y_pred_rf=rf_random.predict(X_test)
print(classification_report(y_test,y_pred_rf))

              precision    recall  f1-score   support

           0       0.62      0.79      0.70       443
           1       0.69      0.49      0.57       416

    accuracy                           0.65       859
   macro avg       0.66      0.64      0.64       859
weighted avg       0.66      0.65      0.64       859



In [38]:
#Prediction and Evaluation the model with CounterVectorizer as input for training phase
y_pred_rfc=rf_random_c.predict(X_test_c)
print(classification_report(y_test,y_pred_rfc))

              precision    recall  f1-score   support

           0       0.64      0.75      0.69       443
           1       0.67      0.55      0.61       416

    accuracy                           0.65       859
   macro avg       0.66      0.65      0.65       859
weighted avg       0.66      0.65      0.65       859



# **Classify and predict in test dataset**

In [39]:
#Read Test dataset with include raw text and labels
#Open test_text 
xt = []
with open('test_text.txt', 'r', encoding='utf-8') as f:
  data = f.readlines()
  
  for row in data:
    xt.append(row)

 #Open test_label
yt = []
with open('test_labels.txt', 'r', encoding='utf-8') as f:
  data = f.read().split('\n')

  for row in data:
    yt.append(row)   

data_t = pd.DataFrame(zip(xt, yt),  columns=['Comments', 'Labels'] )
data_t.head()

,Comments,Labels
0,@user Can U Help?||More conservatives needed o...,0
1,"Just walked in to #Starbucks and asked for a ""...",1
2,#NOT GONNA WIN \n,0
3,@user He is exactly that sort of person. Weird...,0
4,So much #sarcasm at work mate 10/10 #boring 10...,1


In [40]:
#Preprocessing in test file
data_t['Comments'] = data_t['Comments'].apply(lambda x: preprocessing_1(x))
data_t['Comments'] = data_t['Comments'].apply(lambda x: preprocessing_2(x))
data_t['Comments'] = data_t['Comments'].apply(lambda x: stem(x))

data_t.head()

,Comments,Labels
0,user u helpmor conserv need tsu paid post stuf...,0
1,walk starbuck ask tall blond hahahaha ironi,1
2,gon na win,0
3,user exactli sort person weirdo,0
4,sarcasm work mate bore dead mate shit absolut ...,1


In [41]:
#Feature extraction in test dataset by use two method
x_tf_test = tf.transform(data_t['Comments']) #TF-IDF
xc_tf_test = ct.transform(data_t['Comments']) #CounterVectorizer

**Evaluated test dataset**

Support vector machine evaluation

In [46]:
#Evaluate support vector machine with tf-idf approach
y_svc1 = svc1.predict(x_tf_test)
print(classification_report(data_t['Labels'],y_svc1))
f1_score(data_t['Labels'], y_svc1, average='macro')

              precision    recall  f1-score   support

           0       0.68      0.55      0.60       473
           1       0.47      0.60      0.52       311

    accuracy                           0.57       784
   macro avg       0.57      0.57      0.56       784
weighted avg       0.59      0.57      0.57       784



0.5640264757264831

In [47]:
#Evaluate support vector machine with countervectorize approach
y_svc2 = svc2.predict(xc_tf_test)
print(classification_report(data_t['Labels'],y_svc2))
f1_score(data_t['Labels'], y_svc2, average='macro')

              precision    recall  f1-score   support

           0       0.69      0.56      0.62       473
           1       0.48      0.62      0.54       311

    accuracy                           0.59       784
   macro avg       0.59      0.59      0.58       784
weighted avg       0.61      0.59      0.59       784



0.5832195736637615

Logistic regression Evaluation

In [48]:
#Evaluate Logistic regression machine with tf-idf approach
y_log1 = log.predict(x_tf_test)
print(classification_report(data_t['Labels'],y_log1))
f1_score(data_t['Labels'], y_log1, average='macro')

              precision    recall  f1-score   support

           0       0.73      0.60      0.66       473
           1       0.52      0.66      0.58       311

    accuracy                           0.62       784
   macro avg       0.62      0.63      0.62       784
weighted avg       0.64      0.62      0.63       784



0.618476430976431

In [50]:
#Evaluate Logistic regression machine with CounterVectorize approach
y_logc = log_c.predict(xc_tf_test)
print(classification_report(data_t['Labels'], y_logc))
f1_score(data_t['Labels'], y_logc, average='macro')

              precision    recall  f1-score   support

           0       0.69      0.56      0.62       473
           1       0.48      0.62      0.54       311

    accuracy                           0.58       784
   macro avg       0.59      0.59      0.58       784
weighted avg       0.61      0.58      0.59       784



0.5794591668512094

Random forest evaluation

In [51]:
#Evaluate Randomforest with CounterVectorizer approach
y_rfc = rf_random.predict(xc_tf_test)
print(classification_report(data_t['Labels'],y_rfc))
f1_score(data_t['Labels'], y_rfc, average='macro')

              precision    recall  f1-score   support

           0       0.71      0.66      0.69       473
           1       0.54      0.60      0.57       311

    accuracy                           0.64       784
   macro avg       0.63      0.63      0.63       784
weighted avg       0.64      0.64      0.64       784



0.6266835080121699

In [52]:
#Evaluate Randomforest with tf-idf approach
y_rf = rf_random.predict(x_tf_test)
print(classification_report(data_t['Labels'],y_rf))
f1_score(data_t['Labels'], y_rf, average='macro')

              precision    recall  f1-score   support

           0       0.69      0.71      0.70       473
           1       0.54      0.51      0.53       311

    accuracy                           0.64       784
   macro avg       0.62      0.61      0.62       784
weighted avg       0.63      0.64      0.63       784



0.6153777539916154